## Аггрегация разметки датасета ruDetox

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик

Данные для разметки были взяты из публичного теста датасета Russe Detox. Всего в выборке было 800 примеров. 

Датасет представляет из себя пары текстов:
- токсичный текст
- аналогичный текст без признаков токсичности

Датасет фильтровался в трех независимых проектах на платформе Яндекс.Толока. Каждый проект проверял одно свойство ответов публичного теста датасета:

(1) оскорбительность “детоксифицированного” текста

(2) связность “детоксифицированного” текста

(3) совпадение смысла “детоксифицированного” текста со своей токсичной версией.

In [1]:
from eval import load_model, evaluate_style_transfer, load_csv
import pandas as pd
import pickle
import numpy as np
import json
from collections import Counter

C:\Users\artem\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator IsotonicRegression from version 1.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### Проект №1. Проверка связности и грамотности текстов

In [2]:
assignments = pd.read_csv('assignments_from_pool_41793535__21-10-2023.tsv', sep='\t')
assignments.head(1)

,INPUT:idx,INPUT:neutral_comment,INPUT:training_counter,OUTPUT:fluent,GOLDEN:fluent,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,249,Ну и зачем вы сейчас выкинули это видео2015год...,1343,partly,NaN,NaN,NaN,https://platform.toloka.ai/task/41793535/00027...,00027db7ff--652c79695dae053080c93ed9,00027db7ff--652c80ffe013ff63cff0532b,00027db7ff--652c80fee013ff63cff05325,78c9275a4ea4f3a6421a3e5be36dc6a2,APPROVED,2023-10-16T00:17:03.055,2023-10-16T00:18:08.230,2023-10-16T00:18:08.230,0.03


Фильтруем толокеров, которые дали меньше половины корректных ответов на контрольных заданиях.

In [3]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    text = row[1]

    out = row[3]
    
    gold = row[4]

    user = row[11]

    if str(user) != "nan" and str(gold) != "nan":
        if out == gold:
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  372
Bad users: 201


201 из 372 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Теперь нужно оставить только основной пул. Контрольные задания создавались вручную из отбракованных ранее примеров, чтобы не было пересечений с тестсетом. На контрольных заданиях есть `GOLDEN:fluent`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:neutral_comment` содержит NaN.

In [4]:
assignments_no_control = assignments[assignments['GOLDEN:fluent'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:neutral_comment'].notnull()]

Собираем ответы голосования большинством для каждого задания.

In [5]:
from collections import defaultdict

text_dict = defaultdict(list)

for text, user, out in zip(
    assignments_no_control_no_null["INPUT:neutral_comment"], assignments_no_control_no_null["ASSIGNMENT:worker_id"], 
    assignments_no_control_no_null["OUTPUT:fluent"]
    ):
    if user not in bad_users:
        text_dict[text].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

797


In [6]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({4: 293, 3: 292, 2: 108, 5: 83, 1: 21})

Только в 83 заданиях перекрытие составило 5 человек. Ко всем заданиям будем применять правило, что большинство должно составить минимум 3 человека для формирования метки по результатам голосования большинством. В заданиях с перекрытием меньше 3 такое правило автоматически невыполнимо.

In [7]:
preds_full = {}
for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [ans[j][1]['out'] for j in range(len(ans))]
    cnt = Counter(lst)
    most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
    if most >= 3:
        res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
        preds_full[keys[i]] = res

In [8]:
len(preds_full)

561

Отфильтровались 239 заданий.

In [9]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['text',]), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

Для упрощения последующей аггрегации соединим полученную разметку сс оригинальным датасетом.

In [10]:
res_df = pd.read_csv('dataset.csv')
res_df = res_df.rename({'outputs': 'text'}, axis=1)

In [11]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['text'] = res_df['text'].apply(format_text)
preds_full_df['text'] = preds_full_df['text'].apply(format_text)

Делаем left join, чтобы соединить голосование и поригинальные тексты.

In [12]:
new = res_df.merge(preds_full_df, on='text', how='left')

In [13]:
new['lb'].isna().sum()

239

NaN'ы в отфильтрованных 239 заданиях. Сохраняем в отдельную переменную результаты разметки.

In [14]:
lit = new.iloc[:, 1:].reset_index(drop=True).copy()

### Проект №2. Проверка токсичности текстов

Схема работы с данным и третьим проектом аналогична первому.

In [15]:
assignments = pd.read_csv('assignments_from_pool_41793133__17-10-2023.tsv', sep='\t')
assignments.head(1)

,INPUT:idx,INPUT:task1_suite_id,INPUT:neutral_comment,INPUT:training_counter,OUTPUT:toxic,GOLDEN:toxic,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,518,1,Зачем вообще эта дума нужна,1,False,NaN,NaN,NaN,https://platform.toloka.ai/task/41793133/00027...,00027db66d--652c6aa0ac0d3c5e4b2bdcde,00027db66d--652c6d259404dc38e65a3f7a,00027db66d--652c6d259404dc38e65a3f78,6172cce8a382906f77a3d64bafdd3a27,APPROVED,2023-10-15T22:52:21.837,2023-10-15T22:52:43.876,2023-10-15T22:52:43.876,0.03


In [16]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    text = row[2]

    out = row[4]
    
    gold = row[5]

    user = row[12]

    if str(user) != "nan" and str(gold) != "nan":
        if out == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  341
Bad users: 158


158 из 341 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

In [17]:
assignments_no_control = assignments[assignments['GOLDEN:toxic'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:neutral_comment'].notnull()]

In [18]:
from collections import defaultdict

text_dict = defaultdict(list)

for text, user, out in zip(
    assignments_no_control_no_null["INPUT:neutral_comment"], assignments_no_control_no_null["ASSIGNMENT:worker_id"], 
    assignments_no_control_no_null["OUTPUT:toxic"]
    ):
    if user not in bad_users:
        text_dict[text].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

800


In [19]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({4: 355, 5: 235, 3: 183, 2: 25, 1: 2})

В 235 заданиях перекрытие 5. Голосование большинством проводится при наличии минимум 3 человек в таком большинстве.

In [20]:
preds_full = {}
for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [ans[j][1]['out'] for j in range(len(ans))]
    cnt = Counter(lst)
    most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
    if most >= 3:
        res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
        preds_full[keys[i]] = res

In [21]:
len(preds_full)

698

Всего в 102 текстах согласованность не была достигнута.

In [22]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['text',]), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

In [23]:
res_df = pd.read_csv('dataset.csv')
res_df = res_df.rename({'outputs': 'text'}, axis=1)

In [24]:
res_df['text'] = res_df['text'].apply(format_text)
preds_full_df['text'] = preds_full_df['text'].apply(format_text)

In [25]:
new = res_df.merge(preds_full_df, on='text', how='left')

In [26]:
new['lb'].isna().sum()

102

NaN'ы в 102 текстах. Значит, аггрегация проведена корректно и не было потеряно меток.

In [27]:
ins = new.iloc[:, 1:].reset_index(drop=True).copy()

### Проект №3. Проверка смысловой идентичности текстов

In [28]:
assignments = pd.read_csv('assignments_from_pool_41793355__21-10-2023.tsv', sep='\t')
assignments.head(1)

,INPUT:idx,INPUT:toxic_comment,INPUT:task1_suite_id,INPUT:neutral_comment,INPUT:training_counter,OUTPUT:is_match,GOLDEN:is_match,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,160,такого долбоебизма как в киеве не будет,1,"такого, как в киеве, не будет.",198,False,NaN,NaN,NaN,https://platform.toloka.ai/task/41793355/00027...,00027db74b--652c72bba8ebdc7126d23fa7,00027db74b--652c758e9404dc38e65bc5bf,00027db74b--652c758e9404dc38e65bc5bc,d171b547bedef18946541e2a2a6ff829,APPROVED,2023-10-15T23:28:14.348,2023-10-15T23:28:55.483,2023-10-15T23:28:55.483,0.03


In [29]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    text = row[1]

    out = row[5]
    
    gold = row[6]

    user = row[13]

    if str(user) != "nan" and str(gold) != "nan":
        if out == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  172
Bad users: 37


37 из 172 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

In [30]:
assignments_no_control = assignments[assignments['GOLDEN:is_match'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:toxic_comment'].notnull()]

In [31]:
from collections import defaultdict

text_dict = defaultdict(list)

for neut, tox, user, out in zip(
    assignments_no_control_no_null["INPUT:neutral_comment"], assignments_no_control_no_null["INPUT:toxic_comment"], 
    assignments_no_control_no_null["ASSIGNMENT:worker_id"], assignments_no_control_no_null["OUTPUT:is_match"]
    ):
    if user not in bad_users:
        text_dict[(neut, tox)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

800


In [32]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 683, 4: 112, 3: 5})

Только в 117 текстах перекрытие меньше 5. Правило остается прежним: большинство из 3 человек формирует оценку.

In [33]:
preds_full = {}
for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [ans[j][1]['out'] for j in range(len(ans))]
    cnt = Counter(lst)
    most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
    if most >= 3:
        res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
        preds_full[keys[i]] = res

In [34]:
len(preds_full)

789

Отфильтровано всего 11 текстов.

In [35]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['text', 'tox']), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

In [36]:
res_df = pd.read_csv('dataset.csv')
res_df = res_df.rename({'outputs': 'text'}, axis=1)

In [37]:
res_df['text'] = res_df['text'].apply(format_text)
preds_full_df['text'] = preds_full_df['text'].apply(format_text)

In [38]:
new = res_df.merge(preds_full_df, on='text', how='left')

In [39]:
new['lb'].isna().sum()

11

Те же 11 текстов имеют NaN'ы теперь.

In [40]:
sen = new.iloc[:, 1:].reset_index(drop=True).copy()

### Аггрегация результатов разметки

In [41]:
ins = ins.rename({'lb': 'insult'}, axis=1)
sen = sen.rename({'lb': 'sense'}, axis=1)
lit = lit.rename({'lb': 'literacy'}, axis=1)

Объединяем всю разметку по сохраненной колонке оригинальных текстов.

In [42]:
df = ins.merge(sen, on='inputs').merge(lit, on='inputs')

In [43]:
df = df.drop(['text_y', 'tox', 'text'], axis=1).rename({'inputs': 'init', 'text_x': 'rewrite'}, axis=1)

Оставляем только пары текстов с непустой разметкой по всем трем проектам.

In [44]:
sub = df[(df['insult'].notna()) & (df['sense'].notna()) & (df['literacy'].notna())].copy()

In [45]:
df = sub[(sub['insult'] == False) & (sub['sense'] == True) & (sub['literacy'] == 'fluent')].copy()

Оставляем только неоскорбительные, связные и грамотные тексты, которые связаны по смыслу со своими токсичными парами.

In [46]:
df.head(5)

,init,rewrite,insult,sense,literacy
0,пиздеж! температуры горения хватит чтобы её ра...,Враньё! Температуры горения хватит чтобы ее ра...,False,True,fluent
1,а ты чмо там был.ты вообще служил.гандон,А ты там был? Ты вообще служил?,False,True,fluent
2,пиздабол ---- а сам где кормишься ?,а сам где кормишься ?,False,True,fluent
3,"этому сайту я давно не доверяю, пишут разную х...","Этому сайту давно не доверяю, пишут всякую ерунду",False,True,fluent
4,у меня сегодня подобный звонок был. достали го...,у меня сегодня подобный звонок был. Достали.,False,True,fluent


### Получение метрик

Оценка будет производиться на основе трех метрик:
- style accuracy
- meaning preservation
- joint fluency

Каждая метрика получается из предиктов отдельной языковой модели по имеющимся текстам, прошедшим фильтрацию. Поверх отдельных метрик строится финальная метрика - Joint score, которая аггрегирует три предыдущих.

In [47]:
style_model, style_tokenizer = load_model("IlyaGusev/rubertconv_toxic_clf")
meaning_model, meaning_tokenizer = load_model("s-nlp/rubert-base-cased-conversational-paraphrase-v1")
cola_model, cola_tolenizer = load_model("s-nlp/ruRoberta-large-RuCoLa-v1")

In [48]:
inputs = df['init'].tolist()
refs = df['rewrite'].tolist()

Чтобы приблизить оценку к человеческой были обучены специальные калибраторы, которые перевзвешивают получившиеся предикты языковых моделей для получения более честной оценки.

In [49]:
with open("score_calibrations_ru.pkl", "rb") as f:
    style_calibrator = pickle.load(f)
    content_calibrator = pickle.load(f)
    fluency_calibrator = pickle.load(f)

C:\Users\artem\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator IsotonicRegression from version 1.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Делаем замер метрик.

In [50]:
res = evaluate_style_transfer(
    original_texts=inputs,
    rewritten_texts=refs,
    style_model=style_model,
    style_tokenizer=style_tokenizer,
    meaning_model=meaning_model,
    meaning_tokenizer=meaning_tokenizer,
    cola_model=cola_model,
    cola_tokenizer=cola_tolenizer,
    style_target_label=0,
    aggregate=True,
    style_calibration=lambda x: style_calibrator.predict(x[:, np.newaxis]),
    meaning_calibration=lambda x: content_calibrator.predict(x[:, np.newaxis]),
    fluency_calibration=lambda x: fluency_calibrator.predict(x[:, np.newaxis]),
)

Style evaluation


Meaning evaluation
Fluency evaluation
Style accuracy:       0.9482819437980652
Meaning preservation: 0.9316206574440002
Joint fluency:        0.8761930465698242
Joint score:          0.7697036862373352
Scores after calibration:
Style accuracy:       0.7789858852900526
Meaning preservation: 0.7183928820164345
Joint fluency:        0.8623976766936412
Joint score:          0.47682560625592224


`Joint score = 0.477`